In [19]:
!pip install matplotlib torch torchvision

  Using cached torch-2.2.2-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached torchvision-0.17.2-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached torch-2.2.2-cp310-cp310-win_amd64.whl (198.6 MB)
Using cached torchvision-0.17.2-cp310-cp310-win_amd64.whl (1.2 MB)
Using cached filelock-3.13.4-py3-none-any.whl (11 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)


In [20]:
import os
import glob
import torch
import cv2
import numpy as np
from PIL import Image

class YOLOv7Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_paths = sorted(glob.glob(os.path.join(img_dir, '*.jpg')))
        self.label_paths = [os.path.join(label_dir, f.split('/')[-1].split('.')[0] + '.txt') for f in self.img_paths]
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label_path = self.label_paths[idx]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        with open(label_path, 'r') as f:
            lines = f.readlines()

        boxes = []
        classes = []
        for line in lines:
            class_id, x, y, w, h = map(float, line.strip().split(' '))
            boxes.append([x, y, x + w, y + h])
            classes.append(class_id)

        boxes = np.array(boxes, dtype=np.float32)
        classes = np.array(classes, dtype=np.float32)

        if self.transform:
            img, boxes, classes = self.transform((img, boxes, classes))

        return img, boxes, classes

In [23]:
from torchvision.transforms import RandomHorizontalFlip
from torchvision import transforms
from albumentations import Resize, Normalize

data_transforms = transforms.Compose([
    RandomHorizontalFlip(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
dataset = YOLOv7Dataset('lables/images', 'lables/labels', transform=data_transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

Train

In [25]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm



# Initialize model
device = select_device('')
half = device.type != 'cpu'  # half precision only supported on CUDA
model = Model(device)

# Load weights
load_classifier(model, 'yolov7.pt') if device.type != 'cpu' else None

# Initialize optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)

# Initialize scheduler
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

# Define loss function
criterion = torch.nn.MSELoss()

# Set logging
logging = set_logging()

# Train loop
for epoch in range(10):  # 10 epochs
    model.train()
    for images, targets in tqdm(dataloader, desc=f'Epoch {epoch + 1}'):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        pred = model(images)

        # Compute loss
        loss_dict = {}
        for i in range(len(pred)):
            loss_dict[f'loss{i}'] = criterion(pred[i], targets[i]['labels'].float())

        # Backward pass
        optimizer.zero_grad()
        total_loss = sum(loss for loss in loss_dict.values())
        total_loss.backward()

        # Update weights
        optimizer.step()

        # Update scheduler
        scheduler.step()

    # Evaluate on validation set
    model.eval()
    with torch.no_grad():
        for images, targets in tqdm(val_dataloader, desc='Validation'):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            pred = model(images)

            # Compute loss
            loss_dict = {}
            for i in range(len(pred)):
                loss_dict[f'loss{i}'] = criterion(pred[i], targets[i]['labels'].float())

            # Compute metrics
            metrics_dict = {}
            for i in range(len(pred)):
                # Compute metrics for each output
                pass  # Add your code here

            # Accumulate metrics
            for k, v in metrics_dict.items():
                logging[k] += v

    # Print metrics
    for k, v in logging.items():
        print(f'{k}: {v / (len(val_dataloader) * (epoch + 1)):.4f}')

ModuleNotFoundError: No module named 'models'